<h1>Chapter 2 | <b>EXTRA</b> data exercise | <b>Identifying Successful Football Managers in Brazilian Football</b></h1>
<p>In this notebook, I'll be writing the code for the aforementionred case study of the book. The main goal is to observe and learn how to structure the data according to our neeeds, which means applying the concept of <i>tidy data</i> aas well as merging different datasets to get the desired outcome.</p>
<p>Here, our question is: <i>who are the most successful football managers in <b>Brazil</b>?</p>


In [34]:
import os
import sys
import warnings
import yaml
import numpy as np
import pandas as pd
import seaborn as sns
from mizani.formatters import percent_format
from plotnine import *
import basedosdados as bd

warnings.filterwarnings("ignore")

In [35]:
# Current script folder path
current_path = os.getcwd()
dirname = f"{current_path}/data/"
config = f"{current_path}/config/"
# Location folders
data_in = f"{dirname}raw/"
data_out = f"{dirname}clean/"

func = f"{current_path.split('da_data_exercises')[0]}da_case_studies/ch00-tech_prep/"
sys.path.append(func)

In [36]:
with open(f"{config}config.yaml", "r") as f:
    project_id = yaml.safe_load(f)

In [37]:
from py_helper_functions import seq, color

<h2>2. Extract data from Basedosdados' GBQ</h2>

In [38]:
query = """
    SELECT 
  ano_campeonato AS season,
  data AS date,
  time_man AS home_team,
  time_vis AS away_team,
  tecnico_man AS manager_home,
  tecnico_vis AS manager_away,
  gols_man AS goals_home,
  gols_vis AS goals_away
    FROM `basedosdados.mundo_transfermarkt_competicoes.brasileirao_serie_a`
    WHERE ano_campeonato >= 2012
"""

In [39]:
df = bd.read_sql(query, billing_project_id=project_id["bd_id"])

Downloading: 100%|██████████| 4303/4303 [00:00<00:00, 4910.21rows/s]


In [40]:
df.head()

,season,date,home_team,away_team,manager_home,manager_away,goals_home,goals_away
0,2016,2016-09-15,Botafogo,Santos FC,Jair Ventura,Dorival Júnior,0,1
1,2016,2016-09-25,Flamengo,Cruzeiro,Zé Ricardo,Mano Menezes,2,1
2,2016,2016-10-09,Flamengo,Santa Cruz,Zé Ricardo,Doriva,3,0
3,2016,2016-10-23,Flamengo,Corinthians,Zé Ricardo,Oswaldo de Oliveira,2,2
4,2016,2016-10-13,Fluminense,Flamengo,Levir Culpi,Zé Ricardo,1,2


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4303 entries, 0 to 4302
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   season        4303 non-null   Int64 
 1   date          4303 non-null   dbdate
 2   home_team     4303 non-null   object
 3   away_team     4303 non-null   object
 4   manager_home  4113 non-null   object
 5   manager_away  4113 non-null   object
 6   goals_home    4302 non-null   Int64 
 7   goals_away    4302 non-null   Int64 
dtypes: Int64(3), dbdate(1), object(4)
memory usage: 281.7+ KB


In [43]:
df[["goals_home", "goals_away"]].isna().value_counts()

goals_home  goals_away
False       False         4302
True        True             1
dtype: int64

In [44]:
df.dropna(subset=["goals_home", "goals_away"], inplace=True)

In [45]:
# Create "points_home" variable
df["points_home"] = df.apply(
    lambda row: 1 if row["goals_home"] == row["goals_away"] 
    else (3 if row["goals_home"] > row["goals_away"] else 0), 
    axis=1
    )

# Create "points_away" variable
df["points_away"] = df.apply(
    lambda row: 1 if row["goals_home"] == row["goals_away"] 
    else (3 if row["goals_home"] < row["goals_away"] else 0), 
    axis=1
    )

In [46]:
df.head()

,season,date,home_team,away_team,manager_home,manager_away,goals_home,goals_away,points_home,points_away
0,2016,2016-09-15,Botafogo,Santos FC,Jair Ventura,Dorival Júnior,0,1,0,3
1,2016,2016-09-25,Flamengo,Cruzeiro,Zé Ricardo,Mano Menezes,2,1,3,0
2,2016,2016-10-09,Flamengo,Santa Cruz,Zé Ricardo,Doriva,3,0,3,0
3,2016,2016-10-23,Flamengo,Corinthians,Zé Ricardo,Oswaldo de Oliveira,2,2,1,1
4,2016,2016-10-13,Fluminense,Flamengo,Levir Culpi,Zé Ricardo,1,2,0,3
